<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_16_fundamentos_de_rag_con_hugging_face_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Sesión 16 – Fundamentos de RAG con Hugging Face + ChromaDB

## Objetivos

* Comprender los conceptos básicos de RAG y su arquitectura.
* Aprender a transformar texto → embeddings → almacenar en base vectorial.
* Construir un mini-RAG con ChromaDB y Hugging Face embeddings.

---

## Contenido

1. **Concepto de RAG**:

   * Separar *memoria a largo plazo* (vector store) del LLM.
   * Flujo: **Pregunta → Embedding → Recuperación → Contexto → LLM → Respuesta**.

2. **Primer prototipo**:

   * Usar `sentence-transformers` de Hugging Face para embeddings.
   * Guardar y consultar embeddings en **ChromaDB**.
   * Pasar contexto recuperado a un LLM (ej. `transformers` o `openai`).

---

## Demo Código

In [1]:
# ========================
# 1. Instalación de librerías
# ========================
#%%capture
#!pip install -q chromadb sentence-transformers datasets transformers accelerate bitsandbytes gradio
!pip install -q -U wikipedia-api chromadb bitsandbytes gradio


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ========================
# 2. Cargar datos de Wikipedia (Paraguay)
# ========================
import wikipediaapi

# Configurar con user-agent válido
wiki_wiki = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoRAG/1.0 (https://github.com/rubuntu)'
)

paginas = [
    "Paraguay", "Historia de Paraguay", "Geografía de Paraguay",
    "Economía de Paraguay", "Cultura de Paraguay", "Asunción",
    "Guerra de la Triple Alianza", "Guerra del Chaco"
]


docs = []
for titulo in paginas:
    page = wiki_wiki.page(titulo)
    if page.exists():
        docs.append(page.text)
        print(f"Cargado: {titulo}")
print(f"Total documentos cargados: {len(docs)}")

# ========================
# 3. Embeddings y ChromaDB
# ========================
from sentence_transformers import SentenceTransformer
import chromadb

emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = emb_model.encode(docs, batch_size=16, show_progress_bar=True)

client = chromadb.Client()
collection = client.create_collection("paraguay_wiki")

for i, doc in enumerate(docs):
    collection.add(documents=[doc], embeddings=[embeddings[i].tolist()], ids=[str(i)])

print("✅ Base vectorial creada con ChromaDB.")

# ========================
# 4. Cargar Gemma-3n (E4B-IT por defecto)
# ========================
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "google/gemma-3n-e2b-it"   # puedes cambiar a "google/gemma-3n-e4b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

print("✅ Modelo cargado:", model_id)

# ========================
# 5. Función RAG con Gemma-3n
# ========================
def rag_answer(question, top_k=3, max_new_tokens=200):
    # Recuperar contexto
    query_emb = emb_model.encode([question])
    results = collection.query(query_embeddings=query_emb.tolist(), n_results=top_k)
    context = "\n".join(results["documents"][0])

    # Prompt sencillo
    prompt = f"""
Eres un asistente útil.
Responde a la pregunta basándote únicamente en el siguiente contexto.
Si el contexto no contiene la respuesta, di "No lo sé".

Contexto:
{context}

Pregunta: {question}

Respuesta:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to("cuda")
    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # limpiar eco del prompt
    if prompt in answer:
        answer = answer.split(prompt)[-1].strip()

    return answer

# ========================
# 6. Preguntas de ejemplo
# ========================
questions = [
    "¿Cuál es la capital de Paraguay?",
    "¿Qué importancia tiene el río Paraguay?",
    "¿Quién fue Francisco Solano López?",
    "¿Qué papel tuvo Paraguay en la Guerra de la Triple Alianza?",
    "¿Cuáles son los principales productos de exportación de Paraguay?"
]

for q in questions:
    print("\n❓", q)
    print("💡", rag_answer(q))


In [2]:
# ========================
# 1. Dependencias
# ========================
# pip install wikipedia-api sentence-transformers chromadb transformers accelerate bitsandbytes torch

import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
try:
    # Solo importamos si se usa 4-bit
    from transformers import BitsAndBytesConfig
except Exception:
    BitsAndBytesConfig = None

import wikipediaapi
from sentence_transformers import SentenceTransformer
import chromadb

# ========================
# 2. Configuración general
# ========================
USE_4BIT = False  # <- Cambia a True si necesitas 4-bit
MODEL_ID = "google/gemma-3n-e2b-it"   # o "google/gemma-3n-e4b-it"

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    # En GPU preferimos bf16 si está disponible; si no, fp16
    try:
        torch_dtype = torch.bfloat16
    except Exception:
        torch_dtype = torch.float16
else:
    # En CPU, usa float32 para evitar problemas de precisión/compatibilidad
    torch_dtype = torch.float32

print(f"🔧 Dispositivo: {device} | torch_dtype: {torch_dtype}")

# ========================
# 3. Cargar datos de Wikipedia (Paraguay)
# ========================
wiki_wiki = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoRAG/1.0 (https://github.com/rubuntu)'
)

paginas = [
    "Paraguay", "Historia de Paraguay", "Geografía de Paraguay",
    "Economía de Paraguay", "Cultura de Paraguay", "Asunción",
    "Guerra de la Triple Alianza", "Guerra del Chaco"
]

docs = []
for titulo in paginas:
    page = wiki_wiki.page(titulo)
    if page.exists():
        docs.append(page.text)
        print(f"✅ Cargado: {titulo}")
    else:
        print(f"⚠️ No existe: {titulo}")
print(f"📚 Total documentos cargados: {len(docs)}")

# ========================
# 4. Embeddings y ChromaDB
# ========================
emb_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = emb_model.encode(docs, batch_size=16, show_progress_bar=True)

client = chromadb.Client()  # Si quieres persistencia: chromadb.PersistentClient(path="./chroma")
# Si la colección ya existe, intenta obtenerla; si no, créala
try:
    collection = client.get_collection("paraguay_wiki")
    # Si existe, la vaciamos y la recreamos para este ejemplo
    client.delete_collection("paraguay_wiki")
except Exception:
    pass
collection = client.create_collection("paraguay_wiki")

for i, doc in enumerate(docs):
    collection.add(documents=[doc], embeddings=[embeddings[i].tolist()], ids=[str(i)])

print("✅ Base vectorial creada con ChromaDB.")

# ========================
# 5. Cargar Gemma-3n (con o sin 4-bit)
# ========================
if USE_4BIT:
    if BitsAndBytesConfig is None:
        raise RuntimeError("BitsAndBytes no disponible. Instala 'bitsandbytes' y 'transformers>=4.56'.")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    llm = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
    )
    # 🔧 Workaround clave para evitar el RuntimeError (clamp_ sobre pesos 4-bit/uint8)
    if hasattr(llm, "config"):
        llm.config.altup_coef_clip = None
    print(f"✅ Modelo cargado en 4-bit (workaround AltUp): {MODEL_ID}")
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    llm = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch_dtype,
        trust_remote_code=True,
    )
    print(f"✅ Modelo cargado sin 4-bit: {MODEL_ID}")

# Asegurar pad_token_id
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# ========================
# 6. Función RAG con Gemma-3n
# ========================
def rag_answer(question, top_k=3, max_new_tokens=200, temperature=0.3, top_p=0.9):
    # Recuperar contexto
    query_emb = emb_model.encode([question])
    results = collection.query(query_embeddings=query_emb.tolist(), n_results=top_k)

    # results["documents"] es una lista de listas
    retrieved_docs = results.get("documents", [[]])
    context = "\n\n---\n\n".join(retrieved_docs[0]) if retrieved_docs and retrieved_docs[0] else ""

    # Prompt
    prompt = f"""Eres un asistente útil.
Responde a la pregunta basándote únicamente en el siguiente contexto.
Si el contexto no contiene la respuesta, di "No lo sé".

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)
    # Mover tensores al dispositivo correcto si hay GPU
    if device == "cuda":
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    outputs = llm.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Limpiar eco del prompt (si lo hay)
    if prompt in answer:
        answer = answer.split(prompt)[-1].strip()

    return answer.strip()

# ========================
# 7. Preguntas de ejemplo
# ========================
questions = [
    "¿Cuál es la capital de Paraguay?",
    "¿Qué importancia tiene el río Paraguay?",
    "¿Quién fue Francisco Solano López?",
    "¿Qué papel tuvo Paraguay en la Guerra de la Triple Alianza?",
    "¿Cuáles son los principales productos de exportación de Paraguay?"
]

for q in questions:
    print("\n❓", q)
    try:
        print("💡", rag_answer(q))
    except Exception as e:
        print("❌ Error al responder:", e)


🔧 Dispositivo: cuda | torch_dtype: torch.bfloat16
✅ Cargado: Paraguay
✅ Cargado: Historia de Paraguay
✅ Cargado: Geografía de Paraguay
✅ Cargado: Economía de Paraguay
✅ Cargado: Cultura de Paraguay
✅ Cargado: Asunción
✅ Cargado: Guerra de la Triple Alianza
✅ Cargado: Guerra del Chaco
📚 Total documentos cargados: 8


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

✅ Modelo cargado sin 4-bit: google/gemma-3n-e2b-it

❓ ¿Cuál es la capital de Paraguay?
💡 Eres un asistente útil.
Responde a la pregunta basándote únicamente en el siguiente contexto.
Si el contexto no contiene la respuesta, di "No lo sé".

Contexto:
Asunción (en guaraní: Paraguay), oficialmente llamada Ciudad de la Asunción,​ es la capital y ciudad más poblada del Paraguay, ubicada en el centro-oeste de la Región Oriental. Es un municipio de primer orden administrado como distrito capital y no está integrado en ningún departamento. Limita al norte con el río Paraguay que lo separa del departamento de Presidente Hayes y de la Región Occidental; al este y sur con el departamento Central; y al oeste con el río Paraguay, que lo separa de la República Argentina por lo que es una ciudad fronteriza. Fue fundada como Nuestra Señora de la Asunción​ el 15 de agosto de 1537 por Juan de Salazar de Espinosa, lo que la hace una de las ciudades más antiguas de Sudamérica.
De acuerdo a InSight Crime, 

In [ ]:
# ========================
# 7. Interfaz con Gradio
# ========================
import gradio as gr

def ask_question(query):
    return rag_answer(query)

demo = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Escribe tu pregunta sobre Paraguay..."),
    outputs="text",
    title="RAG sobre Paraguay (Wikipedia + LLM)",
    description="Haz preguntas sobre Paraguay usando RAG con ChromaDB y un modelo abierto (Gemma, Mistral, Llama, Qwen)."
)

demo.launch(share=True)

---

## Preguntas de discusión

1. ¿Por qué un LLM necesita un vector store externo para RAG?
2. ¿Qué limitaciones tendría un mini-RAG con pocas docenas de documentos?
3. ¿Cómo escalarías esto a millones de documentos?

---